In [34]:
import pandas as pd

In [35]:
df = pd.read_parquet('sell_in_pred.parquet')
df.head()

,product_id,periodo,primer_periodo,ultimo_periodo,tn_norm,periodo_dt,mes,quarter,month_in_quarter,year,...,prop_product_yearquarter_brand,prop_product_year_cat1,prop_product_year_cat2,prop_product_year_brand,tn_futuro,tn_prediccion,average_tn,std_dev_tn,tn_futuro_real,tn_prediccion_real
0,20001,201912,201701,201912,0.356686,201912,12,4,3,2019,...,24.683874,0.702526,1.252843,6.476876,NaN,-0.890872,1398.344322,298.145460,NaN,1239.079079
1,20002,201912,201701,201912,0.256522,201912,12,4,3,2019,...,4.739874,0.507655,0.905322,1.198157,NaN,-0.553614,1009.368178,303.834835,NaN,919.101355
2,20003,201912,201701,201912,0.011975,201912,12,4,3,2019,...,7.660107,1.258237,1.673822,2.237771,NaN,-0.653311,889.004243,292.036581,NaN,701.710657
3,20004,201912,201701,201912,-0.150213,201912,12,4,3,2019,...,5.474930,0.899303,1.196336,1.599409,NaN,-0.569770,671.615383,224.450085,NaN,510.015118
4,20005,201912,201701,201912,-0.233451,201912,12,4,3,2019,...,5.091663,0.836348,1.112587,1.487444,NaN,-0.508377,644.200514,218.273222,NaN,482.279275


In [36]:
df_fut = pd.read_parquet('sell_in_pred_futuro.parquet')
df_fut.head()

,product_id,periodo,primer_periodo,ultimo_periodo,tn_norm,periodo_dt,mes,quarter,month_in_quarter,year,...,prop_product_yearquarter_brand,prop_product_year_cat1,prop_product_year_cat2,prop_product_year_brand,tn_futuro,tn_prediccion,average_tn,std_dev_tn,tn_futuro_real,tn_prediccion_real
0,20001,201912,201701,201912,0.356686,201912,12,4,3,2019,...,24.683874,0.702526,1.252843,6.476876,NaN,-0.890872,1398.344322,298.145460,NaN,1239.079079
1,20002,201912,201701,201912,0.256522,201912,12,4,3,2019,...,4.739874,0.507655,0.905322,1.198157,NaN,-0.553614,1009.368178,303.834835,NaN,919.101355
2,20003,201912,201701,201912,0.011975,201912,12,4,3,2019,...,7.660107,1.258237,1.673822,2.237771,NaN,-0.653311,889.004243,292.036581,NaN,701.710657
3,20004,201912,201701,201912,-0.150213,201912,12,4,3,2019,...,5.474930,0.899303,1.196336,1.599409,NaN,-0.569770,671.615383,224.450085,NaN,510.015118
4,20005,201912,201701,201912,-0.233451,201912,12,4,3,2019,...,5.091663,0.836348,1.112587,1.487444,NaN,-0.508377,644.200514,218.273222,NaN,482.279275


In [37]:
def perdida(real, prediccion):
    total = real.sum()
    perdida =  abs(real - prediccion).sum()
    resultado = perdida / total
    return resultado

In [38]:
df_pred = df[['product_id', 'tn_futuro_real', 'tn_prediccion_real']]
df_pred

,product_id,tn_futuro_real,tn_prediccion_real
0,20001,NaN,1239.079079
1,20002,NaN,919.101355
2,20003,NaN,701.710657
3,20004,NaN,510.015118
4,20005,NaN,482.279275
...,...,...,...
775,21263,NaN,0.003631
776,21265,NaN,0.033468
777,21266,NaN,0.046271
778,21267,NaN,0.067199


In [39]:
perdida = perdida(df_pred['tn_futuro_real'], df_pred['tn_prediccion_real'])
perdida

C:\Users\waldo\AppData\Local\Temp\ipykernel_29720\1250120186.py:4: RuntimeWarning: invalid value encountered in scalar divide
  resultado = perdida / total


nan

In [40]:
kaggle =  df_fut[['product_id','tn_prediccion_real']]
kaggle['tn_prediccion_real'] = kaggle['tn_prediccion_real'].clip(lower=0)
kaggle.columns = ['product_id', 'tn']
kaggle.to_csv('kaggle-new.csv', index=False)

C:\Users\waldo\AppData\Local\Temp\ipykernel_29720\2128074012.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kaggle['tn_prediccion_real'] = kaggle['tn_prediccion_real'].clip(lower=0)
